In [1]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf

Data Gathering

Download dataset and extract to train_data and val_data.

In [2]:
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
    

--2022-12-13 11:01:52--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 172.217.1.208, 142.250.188.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   174MB/s    in 0.8s    

2022-12-13 11:01:53 (174 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2022-12-13 11:01:53--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 172.217.1.208, 142.250.188.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [a

In [3]:
TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"
!unzip -q horse-or-human.zip -d $TRAIN_DIR
!unzip -q validation-horse-or-human.zip -d $VAL_DIR
     

Read a sample image and check pixel range.



In [4]:
train_sample_fn = os.listdir(os.path.join(TRAIN_DIR, "horses"))[0]
train_sample_path = os.path.join(TRAIN_DIR, "horses", train_sample_fn)
 

In [5]:
img = Image.open(train_sample_path)
img_data = np.asarray(img)

print("Image shape:", img_data.shape)
print("Pixel in range:", np.min(img_data), np.max(img_data))

print(img_data[:1])
     

Image shape: (300, 300, 4)
Pixel in range: 0 255
[[[167 196 249 255]
  [156 189 243 255]
  [150 185 240 255]
  ...
  [125 166 231 255]
  [125 166 231 255]
  [126 166 232 255]]]


Data Flow


In [7]:
img_size = (150, 150)
img_shape = (*img_size, 3)    # We didn't include transparency channel
 

Create data flow from a directory, each image is scaled to the range 0-255.



In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def gen_new_data(data_folder):
  data_gen = ImageDataGenerator(rescale=1./255)
  data = data_gen.flow_from_directory(
      data_folder,
      target_size=img_size,
      batch_size=64,
      class_mode="binary",
  )
  return data
     

train_data_gen = gen_new_data(TRAIN_DIR)
val_data_gen = gen_new_data(VAL_DIR)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


Load Pretrained Model

Load InceptionV3 pretrained on ImageNet and freeze all layers above mixed7. This layer will give us features extracted from InceptionV3. We're gonna replace the top of the model with our own classifier.*italicized text*

In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pretrained_model = InceptionV3(include_top=False, weights="imagenet", input_shape=(150, 150, 3))
     

87910968/87910968 [==============================] - 0s 0us/step


In [10]:
# Freeze all layers
for layer in pretrained_model.layers:
  layer.trainable = False

# We keep all layers up to `mixed7`
last_pretrained_layer = pretrained_model.get_layer(name="mixed7")
print("Last layer shape:", last_pretrained_layer.output_shape)

Last layer shape: (None, 7, 7, 768)


Model Architecture & Training


In [11]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
     

train_model = Flatten()(last_pretrained_layer.output)
train_model = Dense(1024, activation="swish")(train_model)
train_model = Dropout(0.1)(train_model)
train_model = Dense(1, activation="sigmoid")(train_model)
     

model = Model(pretrained_model.input, train_model)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [12]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
model.fit(train_data_gen, epochs=5, validation_data=val_data_gen)
     

Epoch 1/5
17/17 [==============================] - 57s 3s/step - loss: 0.1060 - acc: 0.9494 - val_loss: 0.0452 - val_acc: 0.9961
Epoch 2/5
17/17 [==============================] - 50s 3s/step - loss: 5.2873e-08 - acc: 1.0000 - val_loss: 0.0452 - val_acc: 0.9961
Epoch 3/5
17/17 [==============================] - 50s 3s/step - loss: 2.9400e-08 - acc: 1.0000 - val_loss: 0.0451 - val_acc: 0.9961
Epoch 4/5
17/17 [==============================] - 51s 3s/step - loss: 1.3395e-08 - acc: 1.0000 - val_loss: 0.0450 - val_acc: 0.9961
Epoch 5/5
17/17 [==============================] - 51s 3s/step - loss: 1.6890e-08 - acc: 1.0000 - val_loss: 0.0449 - val_acc: 0.9961


In [13]:
model.evaluate(val_data_gen)


4/4 [==============================] - 9s 2s/step - loss: 0.0449 - acc: 0.9961


[0.0449162982404232, 0.99609375]